# EI ST4

## Imports

In [1]:
import pandas as pd
from datetime import datetime
import math

In [2]:
df = pd.read_csv(r"DS_CentraleSupelec_ST42021/DS_CentraleSupelec_train.csv")

## Cleaning up the dataframe

In [3]:
df

,userId,age,gender,registrationDate,languageCode,countryCode,emailContactable,postalContactable,postalCode,numberChildren,...,detergentType,MrPropreTrier,AntikalTrier,ArielTrier,DashTrier,scentLover,petOwner,ecoInterest,closestShop,washDishes
0,use_XbA1FTDcCrTMNTHK1851TzjyPMP,24,F,2019-10-14T17:33:37Z,fr,FRA,True,True,68540,1,...,Liquid,NaN,NaN,NaN,NaN,NonUser,Yes,NaN,NaN,Auto
1,use_3WHgsMVGSg5MHG2zja91TzdfmY2,67,F,2017-07-25T07:00:31Z,fr,FRA,True,True,17390,3,...,Pods,Known Trier,Known Trier,Known Trier,NaN,NaN,No,High,NaN,Auto
2,use_2SisOiR8QwDaHI4svm11TzcvK7V,45,F,2015-06-25T00:00:00Z,fr,FRA,True,True,30200,1,...,Liquid,NaN,NaN,Known Trier,Known Trier,NonUser,Yes,High,NaN,Auto
3,use_W0T1LmfKaQPJYD1RTWh1Tzdc69H,35,F,2018-01-31T07:24:39Z,fr,FRA,True,True,61400,2,...,Pods,NaN,NaN,Known Trier,Known Trier,Low,Yes,Medium,NaN,Hand
4,use_KwYWZ7UMl4veveOaPGK1TzddtmI,38,F,2016-01-26T11:42:17Z,fr,FRA,True,True,30620,0,...,Liquid,Known Trier,Known Trier,Known Trier,Known Trier,NaN,Yes,Low,NaN,Hand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,use_ICSyEvCcRNPpQeBdWcM1TzdczV8,53,F,2015-03-11T20:03:16Z,fr,FRA,True,True,77240,4+,...,Pods,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Auto
11996,use_Q5DNYeSmG7g6VeMV4n71TzeTRRz,43,F,2015-10-10T19:16:19Z,fr,FRA,True,True,45120,0,...,Liquid,NaN,NaN,NaN,NaN,Low,Yes,Medium,NaN,Hand
11997,use_rEoce5pc0MlKpndfzNj1Tze3J1L,42,F,2015-05-08T02:59:34Z,fr,FRA,True,True,35170,3,...,Liquid,NaN,NaN,NaN,NaN,NaN,Yes,High,NaN,Hand
11998,use_0AuDL2wOJHy9I16zSC21TzeHRMo,39,M,2019-05-20T11:59:02Z,fr,FRA,True,True,45160,1,...,Liquid,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Auto


Get unique count for each variable

In [4]:
df.nunique()

userId               12000
age                     74
gender                   2
registrationDate      9875
languageCode             2
countryCode              1
emailContactable         2
postalContactable        2
postalCode            3794
numberChildren          10
lastActivityDate     12000
reactivationValue        5
toothBrushType           2
detergentType           10
MrPropreTrier            1
AntikalTrier             1
ArielTrier               1
DashTrier                1
scentLover               4
petOwner                 9
ecoInterest              4
closestShop              6
washDishes               2
dtype: int64

Check variable data type

In [5]:
df.dtypes

userId               object
age                   int64
gender               object
registrationDate     object
languageCode         object
countryCode          object
emailContactable       bool
postalContactable      bool
postalCode           object
numberChildren       object
lastActivityDate     object
reactivationValue    object
toothBrushType       object
detergentType        object
MrPropreTrier        object
AntikalTrier         object
ArielTrier           object
DashTrier            object
scentLover           object
petOwner             object
ecoInterest          object
closestShop          object
washDishes           object
dtype: object

We can remove the `languageCode` and `countryCode` column as they are constant

In [6]:
df[~df.languageCode.isin(['fr', 'FR'])].empty and df[~df.countryCode.eq('FRA')].empty

True

In [7]:
df = df.drop(columns=['languageCode', 'countryCode'])

We will also remove the `userId`, `registrationDate`, `reactivationValue`, `emailContactable` and `postalContactable` as they are irrelevant. `postalCode` as it will be to hard to analyse.

In [8]:
df = df.drop(columns=['userId', 'registrationDate', 'reactivationValue', 'emailContactable', 'postalContactable', 'postalCode'])

We are going to replace the `washDishes` `STRING` column with an `INT` column

In [9]:
df["washDishes"] = df["washDishes"].apply(lambda e: 1 if e == 'Auto' else -1)

In the `MrPropreTrier`, `AntikalTrier`, `ArielTrier`, `DashTrier` we will replace `Known Trier` with `1` and `Nan` with `-1`

In [10]:
for c in ['MrPropreTrier', 'AntikalTrier', 'ArielTrier', 'DashTrier']:
    df[c[:-5]] = df.apply(lambda row: 1 if row[c] == 'Known Trier' else -1, axis=1)
    df = df.drop(columns=[c])

We will replace `detergentType` by `liquid`, `pods` and `powder` 

In [11]:
# pods
df['pods'] = df['detergentType'].apply(lambda e: 1 if pd.notna(e) and 'Pods' in e else -1)

# powder
df['powder'] = df['detergentType'].apply(lambda e: 1 if pd.notna(e) and 'Powder' in e else -1)

# liquid
df['liquid'] = df['detergentType'].apply(lambda e: 1 if pd.notna(e) and 'Liquid' in e else -1)

# removes extra column
df = df.drop(columns=['detergentType']) 

Replacing `toothBrushType` with `electricToothbrush`

In [12]:
df['electricToothbrush'] = df.apply(lambda row: 1 if row['toothBrushType'] == 'Electric' else -1, axis=1)
df = df.drop(columns=['toothBrushType'])

Replacing `petOwner` with `hasPet`, if the first 3 characters are 'Yes' or 'Oui' the value is `1`, else it is `-1`

In [13]:
def likesPets(s:str) -> float:
    if s in ['Yes, we love our furry friends', "Oui, j'adore nos petites boules de poils !"]: 
        return 1

    if s in ['Yes']: 
        return 0.9

    if s in ["Non, j'aime les animaux, mais je n'en ai pas pour le moment.", 'No, we love animals but no pets here for now']:   
        return 0.8
    
    if s in ['No - maybe future']:
        return 0.5

    if s in ['No']:
        return 0.3

    if s in ['Des animaux dans la maison ? Non merci !', "No, we'd never have pets in the house"]:
        return 0

# creating a new column for animal lovers
df['likesPets'] = df['petOwner'].apply(likesPets)

# creating a new column for pet owners
df['hasPet'] = df['petOwner'].apply(lambda e: 1 if str(e)[:3] == 'Yes' or str(e)[:3] == 'Oui' else -1)

df = df.drop(columns=['petOwner'])

Transforms a `ISO 8601` time string to the amount of days since the time string

In [14]:
def timeStringToDelta(timeString: str) -> int:
    if pd.isna(timeString): return None
    return int((datetime.utcnow().timestamp() - datetime.strptime(timeString[:10], "%Y-%m-%d").timestamp()) / 3600 / 24)

Replaces `lastActivityDate` with `daysSinceActivity`: an `integer` representing the amount of ellapsed days since last activity

In [15]:
df['daysSinceActivity'] = df['lastActivityDate'].apply(timeStringToDelta)

# normalizess the new column
minV = df['daysSinceActivity'].min()
maxV = df['daysSinceActivity'].max()
df['daysSinceActivity'] = df['daysSinceActivity'].apply(lambda e: (e - minV) / (maxV - minV))

# replaces Nan by the avg
df['daysSinceActivity'].fillna(df['daysSinceActivity'].mean(skipna=True))

# removes the extra column
df = df.drop(columns=['lastActivityDate'])

The possible values for `numberChildren` are `'1'`, `'2'`, `'3'`, `'4'`, `'4+'` and `'NaN'`. We will be creating a column `nbChildren` of type `int` where `'NaN'` will be mapped to `None`.

In [16]:
def childrenMagik(children: str) -> float:
    if children == '0': return 0
    if children == '1': return 1 / 5
    if children == '2': return 2 / 5
    if children == '3': return 3 / 5
    if children == '4': return 4 / 5
    if children == '4+': return 1
    return None

df['nbChildren'] = df['numberChildren'].apply(childrenMagik)

# Replacing nan with the avg
df['nbChildren'] = df['nbChildren'].fillna(df['nbChildren'].mean(skipna=True))

df = df.drop(columns=['numberChildren'])

KeyError: 'nbChildren'

Replaces `F` with `1` and `M` with `-1` in the `gender` column

In [18]:
df['gender'] = df['gender'].apply(lambda e: 1 if e == "F" else -1)

Changes `ecoInterest` replacing `High`, `Medium`, `Low`, `Very high` with a scale going from `0` to `1`. We the replace `nan` with the avg.

In [19]:
def textScaleToFloat(s:str) -> float:
    if s == 'Very high': return 1
    if s == 'High': return 2 / 3
    if s == 'Medium': return 1 / 3
    if s == 'Low': return 0
    return None

# replaces the 'normal' values with floats
df['ecoInterest'] = df['ecoInterest'].apply(textScaleToFloat)

# replaces the nan with the avg
df['ecoInterest'] = df['ecoInterest'].fillna(df['ecoInterest'].mean(skipna=True))

Changes `scentLover` replacing `NonUser`, `Low`, `Medium`, `High` with a scale going from 0 to 1. We the replace `nan` with the avg.

In [20]:
def textScaleToFloat(s:str) -> float:
    if s == 'High': return 1
    if s == 'Medium': return 2 / 3
    if s == 'Low': return 1 / 3
    if s == 'NonUser': return 0
    return None

# replaces the 'normal' values with floats
df['scentLover'] = df['scentLover'].apply(textScaleToFloat)

# replaces the nan with the avg
df['scentLover'] = df['scentLover'].fillna(df['scentLover'].mean(skipna=True))

In [21]:
df

,age,gender,scentLover,ecoInterest,closestShop,washDishes,MrPropre,Antikal,Ariel,Dash,pods,powder,liquid,electricToothbrush,likesPets,hasPet,daysSinceActivity,nbChildren
0,24,1,0.000000,0.388626,NaN,1,-1,-1,-1,-1,-1,-1,1,-1,0.9,1,308,1.0
1,67,1,0.378981,0.666667,NaN,1,1,1,1,-1,1,-1,-1,-1,0.3,-1,28,3.0
2,45,1,0.000000,0.666667,NaN,1,-1,-1,1,1,-1,-1,1,-1,0.9,1,130,1.0
3,35,1,0.333333,0.333333,NaN,-1,-1,-1,1,1,1,-1,-1,-1,0.9,1,340,2.0
4,38,1,0.378981,0.000000,NaN,-1,1,1,1,1,-1,-1,1,1,0.9,1,158,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,53,1,0.378981,0.388626,NaN,1,-1,-1,-1,-1,1,-1,-1,-1,0.9,1,10,5.0
11996,43,1,0.333333,0.333333,NaN,-1,-1,-1,-1,-1,-1,-1,1,1,0.9,1,167,0.0
11997,42,1,0.378981,0.666667,NaN,-1,-1,-1,-1,-1,-1,-1,1,-1,0.9,1,33,3.0
11998,39,-1,0.378981,0.388626,NaN,1,-1,-1,-1,-1,-1,-1,1,1,0.9,1,260,1.0
